# initial attempts at converting cleo data to hdf5

In [1]:
import cleo_tools as cleo
import numpy as np
import h5py
from math import sqrt

import matplotlib.pylab as plt
import seaborn as sn

%matplotlib notebook

In [2]:
#for i in range(0,1):
#    collision = collisions[i]
#    pions,kaons,muons,electrons,photons = collision
#    print len(pions)

def mass(e,px,py,pz):
    m_i = (e**2)+(px**2)+(py**2)+(pz**2)
    m = sqrt(m_i)
    return m

In [3]:
infilename = 'data31_100k_LARGE.dat'
#infilename = 'data31_500k.dat'
#infilename = 'data31.dat'


infile = open(infilename)
collisions = cleo.get_collisions(infile)


lp = []
lk = []
lm = []
le = []
lph = []

pis = []
kas = []
mus = []
els = []
phs = []

for collision in collisions:
    
    pions,kaons,muons,electrons,photons = collision
    
    npi = len(pions)
    nka = len(kaons)
    nmu = len(muons)
    nel = len(electrons)
    nph = len(photons)
    
    lp.append(npi)
    lk.append(nka)
    lm.append(nmu)
    le.append(nel)
    lph.append(nph)
    
    for pion in pions:
        
        pis.append(pion)
    
    for kaon in kaons:
        
        kas.append(kaon)
        
    for muon in muons:
        
        mus.append(muon)
    
    for electron in electrons:
        
        els.append(electron)
    
    for photon in photons:
        
        phs.append(photon)
    
    #print "event %i: |" % (i), "pions: %i | kaons: %i | muons: %i | electrons: %i | photons: %i | " % (npi,nka,nmu,nel,nph)


In [5]:
alg = 'lzf'

outfilename = infilename.replace('.dat','.hdf5')

with h5py.File(outfilename,'a') as hf:
    hf.create_dataset('npions',data=lp)
    hf.create_dataset('nkaons',data=lk)
    hf.create_dataset('nmuons',data=lm)
    hf.create_dataset('nelectrons',data=le)
    hf.create_dataset('nphotons',data=lph)
    
    g1 = hf.create_group('particles')
    
    g1.create_dataset('pions',data=pis)
    g1.create_dataset('kaons',data=kas)
    g1.create_dataset('muons',data=mus)
    g1.create_dataset('electrons',data=els)
    g1.create_dataset('photons',data=phs)

In [6]:
'''
pis2 = []
pos = []

with h5py.File('cleo_test.hdf5','r') as hf:
    
    npions = hf.get('npions')
    
    pions = hf.get('particles/pions')
    
    j = 0
    
    for i in npions:
        j += i
        pos.append(j)
    
    for k in range(0,len(npions)):
        pis2.append(pions[k:k+1])
''';

In [7]:
# Bellis edits

pis2 = []
pos = []

hist_val = []

with h5py.File('cleo_test.hdf5','r') as hf:
    
    npions = hf.get('npions')
    pions = hf.get('particles/pions')

    nkaons = hf.get('nkaons')
    kaons = hf.get('particles/kaons')

    
    pi_start = 0
    k_start = 0
    
    for nk,np in zip(nkaons,npions):
        pi_end = pi_start + np
        k_end = k_start + nk
        #print pi_start,pi_end
        event_pions = pions[pi_start:pi_end]
        event_kaons = kaons[k_start:k_end]
        
        pi_start = pi_end
        k_start = k_end
        
        #print event_pions
        for ep in event_pions:
            e = ep[0]
            hist_val.append(e)
            
            

        for ek in event_kaons:
            e = ek[0]
            hist_val.append(e)
        
            
            
#plt.hist(hist_val)

In [8]:
# FUCK

In [9]:
m2 = []
pi = []
ka = []
mu = []
el = []
ph = []
import time

t_i = time.time()

infile = open('data31_100k_LARGE.dat')
collisions = cleo.get_collisions(infile)

for collision in collisions:
    pions,kaons,muons,electrons,photons=collision
    for pion in pions:
        e,px,py,pz = pion[0:4]
        pi.append(mass(e,px,py,pz))
    for kaon in kaons:
        e,px,py,pz = kaon[0:4]
        ka.append(mass(e,px,py,pz))
    for muon in muons:
        e,px,py,pz = muon[0:4]
        mu.append(mass(e,px,py,pz))
    for electron in electrons:
        e,px,py,pz = electron[0:4]
        el.append(mass(e,px,py,pz))
    for photon in photons:
        e,px,py,pz = photon[0:4]
        ph.append(mass(e,px,py,pz))

t_f = time.time() - t_i
print t_f

8.40094614029


In [10]:
masses = []

pion = []
kaon = []
muon = []
electron = []
photon = []

start = time.time()
with h5py.File('cleo_test_large_c_lzf.hdf5','r') as hf:
#with h5py.File('cleo_test.hdf5','r') as hf:

    
    npions = hf.get('npions').value
    nkaons = hf.get('nkaons').value
    nmuons = hf.get('nmuons').value
    nelecs = hf.get('nelectrons').value
    nphots = hf.get('nphotons').value
    
    pions = hf.get('particles/pions').value
    kaons = hf.get('particles/kaons').value
    muons = hf.get('particles/muons').value
    elecs = hf.get('particles/electrons').value
    phots = hf.get('particles/photons').value
    
    print "Read in the all the data in %f seconds" % (time.time()-start)

    t_i = time.time()
    
    #nlist = [npions,nkaons,nmuons,nelecs,nphots]
    nlist = [npions]
    plist = [pions,kaons,muons,elecs,phots]
    elist = [pion,kaon,muon,electron,photon]
    
    for i in range(0,len(nlist)):
        
        n_particles = nlist[i]
        particles = plist[i]
        outlist = elist[i]
        n_start = 0
        
        for j in n_particles:
            
            n_end = n_start + j
            #print n_start,n_end

            event_particles = particles[n_start:n_end]
            #print event_particles
            for k in event_particles:
                
                e,px,py,pz = k[0:4]
                
                outlist.append(mass(e,px,py,pz))
            
            n_start = n_end

t_f = time.time()-t_i


print t_f

IOError: Unable to open file (Unable to open file: name = 'cleo_test_large_c_lzf.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
hf = h5py.File('cleo_test.hdf5','r')

pions = hf.get('particles/pions')

In [ ]:
type(pions)

In [ ]:
type(pions.value)